# PREPARACION DE LA DATA

### Importar librerias

In [53]:
import pandas as pd
import requests
import time 
import random
pd.options.mode.chained_assignment = None

In [54]:
resumen = "https://riskzone.anywhereportfolio.com.ar:9099/api/parametria/miembrocompensador/getmiembrocompensadorparticipantecategoriaresumen"
detalle = "https://riskzone.anywhereportfolio.com.ar:9099/api/parametria/miembrocompensador/getmiembrocompensadorparticipantecategoriadetalle"


tokenobj = {'key': 'value'}
req1 = requests.get(resumen, json = tokenobj)
resu  = (req1.json())


req2 = requests.get(detalle, json = tokenobj)
deta  = (req2.json())


In [55]:
resu["Footer"][2]
deta["Details"][66]

{'MiembroCompensadorID': 715,
 'MiembroCompensadorCodigo': '330',
 'MiembroCompensadorDescripcion': 'YPF SOCIEDAD ANONIMA',
 'RubroParticipanteID': 0,
 'RubroParticipanteDescripcion': '',
 'CategoriaRiesgoID': 2,
 'CategoriaRiesgoDescripcion': 'B'}

In [56]:

def format_float(value):
    return f'{value:,.2f}'

pd.options.display.float_format = format_float

url_saldos_totales = "https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados"


In [57]:
tokenobj = {'key': 'value'}
REQUEST_GENERAL = requests.get(url_saldos_totales, json = tokenobj)
data  = (REQUEST_GENERAL.json())
df = pd.DataFrame(data)


In [58]:
df.columns

Index(['MiembroCompensadorID', 'CuentaCompensacionID', 'FinalidadID',
       'MonedaID', 'MonedaDescripcion', 'Cotizacion',
       'MiembroCompensadorCodigo', 'CuentaCompensacionCodigo',
       'FinalidadDescripcion', 'EgresoNoVerificado', 'IngresoNoVerificado',
       'EgresoNoVerificadoMoneda', 'IngresoNoVerificadoMoneda', 'DRPRequerido',
       'DRPRequeridoMoneda', 'SpotRequerido', 'SpotRequeridoMoneda',
       'CuentaNeteoID', 'CuentaNeteoCodigo', 'SaldoInicial',
       'EgresoVerificado', 'IngresoVerificado', 'SaldoInicialMoneda',
       'EgresoVerificadoMoneda', 'IngresoVerificadoMoneda', 'CuitCuil',
       'CuentaNeteoDescripcion', 'MargenRequeridoAnterior',
       'MargenRequeridoDelDia', 'MargenRequeridoTotal', 'EntidadBursatilID',
       'EntidadBursatilCodigo', 'EntidadBursatilDescripcion',
       'MargenRequeridoAnteriorSC', 'HasCuentaInfoEmpty'],
      dtype='object')

### PRIMER FILTRO: Dejar solo finalidad márgenes

In [59]:
df = df[(df.FinalidadID == 2)]


### Definir saldos finales y crear columnas de LOP


 en saldos finalidades este saldo inicial se llama SALDO NETO INTEGRADO

In [60]:
df["SaldoInicialAP5"] = df["SaldoInicial"] + df["MargenRequeridoAnterior"]
df["SaldoFinalVerificado"] = df["SaldoInicialAP5"] + df["IngresoVerificado"] + df["EgresoVerificado"] #+ df["MargenRequeridoTotal"]
df["SaldoFinalNoVerificado"] = df["SaldoFinalVerificado"] + df["IngresoNoVerificado"] + df["EgresoNoVerificado"]
df = df[["MiembroCompensadorCodigo","MiembroCompensadorID","CuentaCompensacionCodigo","CuentaCompensacionID","CuentaNeteoCodigo","MonedaDescripcion","SaldoInicialAP5","SaldoFinalVerificado","SaldoFinalNoVerificado","MargenRequeridoDelDia"]]


In [61]:
df

,MiembroCompensadorCodigo,MiembroCompensadorID,CuentaCompensacionCodigo,CuentaCompensacionID,CuentaNeteoCodigo,MonedaDescripcion,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia
0,320,694,1001,1001,2001,Pesos,"1,062,891.23","1,062,891.23","1,062,891.23","-2,566.78"
1,320,694,1001,1001,3035,Pesos,"150,123.63","150,123.63","150,123.63","-6,710.49"
2,320,694,1001,1001,42379,Pesos,"6,018.00","6,018.00","6,018.00",48.56
3,320,694,1001,1001,45575,Pesos,"1,007,254.66","1,007,254.66","1,007,254.66","59,988.34"
4,320,694,1001,1001,95818,Pesos,"1,778,980.00","1,778,980.00","1,778,980.00",0.00
...,...,...,...,...,...,...,...,...,...,...
7247,340,726,1340,14751,24943,Pesos,664.02,664.02,664.02,0.00
7277,349,736,1349,14778,92143,Pesos,"188,592,603.44","189,613,169.81","189,613,169.81",0.00
7281,351,739,30586,14790,30586,Pesos,"25,726,100.00","25,726,100.00","25,726,100.00","-57,400,000.00"
7296,365,756,1365,14885,168031,Pesos,"262,723.23","262,723.23","262,723.23",0.00


In [62]:

agg_functions = {'MiembroCompensadorCodigo': 'first', 'CuentaCompensacionCodigo': 'first', 'SaldoInicialAP5': 'sum', "SaldoFinalVerificado":"sum","SaldoFinalNoVerificado":"sum", "MargenRequeridoDelDia":"sum"}
df_pesos = df.groupby(df['CuentaNeteoCodigo']).aggregate(agg_functions).reset_index()
df = df_pesos
df = df[["MiembroCompensadorCodigo","CuentaCompensacionCodigo","CuentaNeteoCodigo","SaldoInicialAP5","SaldoFinalVerificado","SaldoFinalNoVerificado","MargenRequeridoDelDia"]]

df["LOP_Verificado"] = df["SaldoFinalVerificado"] + df["MargenRequeridoDelDia"]
df["LOP_NoVerificado"] = df["SaldoFinalNoVerificado"] + df["MargenRequeridoDelDia"]

In [63]:
saldo_negativo = df

In [64]:
saldo_negativo

,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado
0,320,1001,1001,"1,990,200.40","1,990,200.40","1,990,200.40",0.00,"1,990,200.40","1,990,200.40"
1,345,1345,100115,"320,158.75","320,158.75","320,158.75",0.00,"320,158.75","320,158.75"
2,269,1269,100240,"67,087.41","67,087.41","67,087.41",0.00,"67,087.41","67,087.41"
3,345,1345,100244,"4,089,285.40","4,089,285.40","4,089,285.40",0.00,"4,089,285.40","4,089,285.40"
4,405,1405,100255,"9,799.88","9,799.88","9,799.88",0.00,"9,799.88","9,799.88"
...,...,...,...,...,...,...,...,...,...
4534,269,1269,99961,207.74,207.74,207.74,0.00,207.74,207.74
4535,378,1378,99968,"225,448.41","225,448.41","225,448.41","-18,899.47","206,548.93","206,548.93"
4536,269,1269,99976,"6,753.37","33,108.19","33,108.19",0.00,"33,108.19","33,108.19"
4537,345,1345,99978,"48,555.00","48,555.00","48,555.00",0.00,"48,555.00","48,555.00"


### Get tipo de cuenta y nombre de MC

In [65]:
url = "https://riskzone.anywhereportfolio.com.ar:9099/api/cuentas/getcuentacompensacion"
df = requests.get(url, verify=False)
data_cuentas  = (df.json())
data_cuentas = pd.DataFrame(data_cuentas)

def get_tipo_de_cuenta(CIMCODIGO):
    CIMCODIGO = str(CIMCODIGO)
    data_mc = ((data_cuentas[data_cuentas["CuentaCompensacionCodigo"] == CIMCODIGO]))
    tipocuenta = data_mc["TipoCuentaCompensacionDescripcion"].iloc[0] #.to_string(Name=False,dtype=False,index=False)
    return(tipocuenta)

def get_mc_name(MC):
    MC = str(MC)
    data_mc = ((data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == MC]))
    name_mc = data_mc["MiembroCompensadorDescripcion"].iloc[0] #.to_string(Name=False,dtype=False,index=False)
    return(name_mc)

C:\Users\aggonzalez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'riskzone.anywhereportfolio.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [66]:
tiposdecuenta = []
for i in range(len(saldo_negativo)):
    CIMCODIGO = (saldo_negativo.iloc[i,1])
    tipo = (get_tipo_de_cuenta(str(CIMCODIGO)))
    tiposdecuenta.append(tipo)
saldo_negativo["TipoDeCuenta"] = tiposdecuenta

nombreMC = []

for i in range(len(saldo_negativo)):
        MC_COD = (saldo_negativo.iloc[i,0])
        nombre_mc = get_mc_name(MC_COD)
        nombreMC.append(nombre_mc)

saldo_negativo["ALyC"] = nombreMC


first_column = saldo_negativo.pop('ALyC')
  
# insert column using insert(position,column_name,first_column) function
saldo_negativo.insert(0, 'ALyC', first_column)


In [67]:
saldo_negativo

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,TipoDeCuenta
0,ROSARIO VALORES SA,320,1001,1001,"1,990,200.40","1,990,200.40","1,990,200.40",0.00,"1,990,200.40","1,990,200.40",AN
1,BALANZ CAPITAL VALORES S A U,345,1345,100115,"320,158.75","320,158.75","320,158.75",0.00,"320,158.75","320,158.75",Terceros
2,ECO VALORES S.A.,269,1269,100240,"67,087.41","67,087.41","67,087.41",0.00,"67,087.41","67,087.41",Terceros
3,BALANZ CAPITAL VALORES S A U,345,1345,100244,"4,089,285.40","4,089,285.40","4,089,285.40",0.00,"4,089,285.40","4,089,285.40",Terceros
4,BULL MARKET BROKERS SA,405,1405,100255,"9,799.88","9,799.88","9,799.88",0.00,"9,799.88","9,799.88",Terceros
...,...,...,...,...,...,...,...,...,...,...,...
4534,ECO VALORES S.A.,269,1269,99961,207.74,207.74,207.74,0.00,207.74,207.74,Terceros
4535,BRIO VALORES AGENTE DE LIQUIDACION Y COMPENSAC...,378,1378,99968,"225,448.41","225,448.41","225,448.41","-18,899.47","206,548.93","206,548.93",Terceros
4536,ECO VALORES S.A.,269,1269,99976,"6,753.37","33,108.19","33,108.19",0.00,"33,108.19","33,108.19",Terceros
4537,BALANZ CAPITAL VALORES S A U,345,1345,99978,"48,555.00","48,555.00","48,555.00",0.00,"48,555.00","48,555.00",Terceros


### Obtener cual es la cuenta propia del MC

In [68]:
url = "https://riskzone.anywhereportfolio.com.ar:9099/api/cuentas/getcuentacompensacion"
r = requests.get(url, verify=False)
data_cuentas  = (r.json())
data_cuentas = pd.DataFrame(data_cuentas)


def get_cim_propia(MC):
    MC = str(MC)
    data_mc = ((data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == (MC)]))
    propia = data_mc[data_mc["EsCuentaPropia"] == True]
    cim_codigo_propia = (propia["CuentaCompensacionCodigo"].to_string(name=False,dtype=False,index=False))
    return(cim_codigo_propia)


C:\Users\aggonzalez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'riskzone.anywhereportfolio.com.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### Crear un DF con las propias y mergearlo con el DF Madre


In [69]:
propiastest = data_cuentas[(data_cuentas.EsCuentaPropia == True)]
propiastest = propiastest[["MiembroCompensadorCodigo", "CuentaCompensacionCodigo"]]
propiastest = (propiastest.rename(columns = {'CuentaCompensacionCodigo':'CuentaPropiaMc'})).reset_index()
propiastest = propiastest[["MiembroCompensadorCodigo", "CuentaPropiaMc"]]

saldo_negativo = saldo_negativo.merge(propiastest, on = "MiembroCompensadorCodigo", how = "left")

### definir el saldo de la propia como el lop verificado de esa cuenta - luego mergear


In [70]:
listado = saldo_negativo[["CuentaCompensacionCodigo","TipoDeCuenta"]]
listado = listado[listado.TipoDeCuenta == "Propia"]
listado
saldlist = []
for cuenta in listado.CuentaCompensacionCodigo:
    saldopropio = saldo_negativo[saldo_negativo.CuentaCompensacionCodigo == cuenta]
    saldodelapropia = (saldopropio["LOP_Verificado"].iloc[0])
    saldlist.append(saldodelapropia)
listado["LOP_Propia"] = saldlist

In [71]:
listado = listado[["CuentaCompensacionCodigo", "LOP_Propia"]].reset_index(drop=True)
listado = listado.rename(columns={"CuentaCompensacionCodigo":"CuentaPropiaMc"})

In [72]:
listado = pd.DataFrame(listado.groupby("CuentaPropiaMc").sum())

In [73]:
listado.reset_index(inplace=True)

In [74]:
len(listado)

141

In [75]:
saldo_negativo = saldo_negativo.merge(listado, how="left", on="CuentaPropiaMc")


In [76]:
len(saldo_negativo)

4539

In [77]:
saldo_negativo = saldo_negativo.fillna(0)

In [78]:
saldo_negativo

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,TipoDeCuenta,CuentaPropiaMc,LOP_Propia
0,ROSARIO VALORES SA,320,1001,1001,"1,990,200.40","1,990,200.40","1,990,200.40",0.00,"1,990,200.40","1,990,200.40",AN,5976,0.00
1,BALANZ CAPITAL VALORES S A U,345,1345,100115,"320,158.75","320,158.75","320,158.75",0.00,"320,158.75","320,158.75",Terceros,26979,"12,746,909.20"
2,ECO VALORES S.A.,269,1269,100240,"67,087.41","67,087.41","67,087.41",0.00,"67,087.41","67,087.41",Terceros,6602,"14,218,250.48"
3,BALANZ CAPITAL VALORES S A U,345,1345,100244,"4,089,285.40","4,089,285.40","4,089,285.40",0.00,"4,089,285.40","4,089,285.40",Terceros,26979,"12,746,909.20"
4,BULL MARKET BROKERS SA,405,1405,100255,"9,799.88","9,799.88","9,799.88",0.00,"9,799.88","9,799.88",Terceros,9425,"38,005,891.52"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4534,ECO VALORES S.A.,269,1269,99961,207.74,207.74,207.74,0.00,207.74,207.74,Terceros,6602,"14,218,250.48"
4535,BRIO VALORES AGENTE DE LIQUIDACION Y COMPENSAC...,378,1378,99968,"225,448.41","225,448.41","225,448.41","-18,899.47","206,548.93","206,548.93",Terceros,54800,"4,008,000.00"
4536,ECO VALORES S.A.,269,1269,99976,"6,753.37","33,108.19","33,108.19",0.00,"33,108.19","33,108.19",Terceros,6602,"14,218,250.48"
4537,BALANZ CAPITAL VALORES S A U,345,1345,99978,"48,555.00","48,555.00","48,555.00",0.00,"48,555.00","48,555.00",Terceros,26979,"12,746,909.20"


In [79]:
len(saldo_negativo)

4539

In [80]:
saldo_negativo[(saldo_negativo.LOP_Propia == 0) & (saldo_negativo.TipoDeCuenta == "Propia")]

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,TipoDeCuenta,CuentaPropiaMc,LOP_Propia
1159,DMA BROKER S.A.,360,37867,161943,0.00,0.00,0.00,0.00,0.00,0.00,Propia,37867,0.00
2353,DMA BROKER S.A.,360,37867,37867,"207,737.60","207,737.60","207,737.60",0.00,"207,737.60","207,737.60",Propia,37867,0.00


In [81]:
len(saldo_negativo[(saldo_negativo.LOP_Propia == 0)].ALyC.unique())

58

### CHECKPOINT - DATA MADRE -> SALDOS y seguimos con saldo_negativo


In [82]:
data_madre = saldo_negativo

### Filtrar cuentas con LOP negativo y aca empieza el analisis

In [83]:
saldo_negativo=saldo_negativo[(saldo_negativo["LOP_Verificado"] < 0)]

In [84]:
saldo_negativo

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,TipoDeCuenta,CuentaPropiaMc,LOP_Propia
37,BARRILLI SOCIEDAD ANONIMA,53,1053,101242,"-22,787,318.95","-2,488,756.95","-2,488,756.95",0.00,"-2,488,756.95","-2,488,756.95",Terceros,101930,0.00
39,AGRINTER CEREALES S R L,526,1526,101274,"-535,275.66","-535,275.66","-535,275.66","2,327.06","-532,948.61","-532,948.61",Terceros,34797,"353,725,230.00"
40,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,101276,"-2,312,040.00","-2,312,040.00","-2,312,040.00",0.00,"-2,312,040.00","-2,312,040.00",Terceros,2021,"1,206,339,191.37"
53,NASINI SOCIEDAD ANONIMA,218,1218,101567,"1,675,770.84","1,681,804.40","1,681,804.40","-13,444,600.00","-11,762,795.60","-11,762,795.60",Terceros,4729,"365,673.55"
57,MAX CAPITAL S.A.,334,1334,101699,"-297,000,000.00","-297,000,000.00","-297,000,000.00",0.00,"-297,000,000.00","-297,000,000.00",Terceros,22558,"182,366,620.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4415,DIAZ RIGANTI CEREALES SRL,131,1131,96112,"-394,973.50","-394,973.50","-394,973.50",0.00,"-394,973.50","-394,973.50",Terceros,9980,"77,068,000.00"
4422,ROAGRO SA,42,1042,9633,"-427,190.02","-427,190.02","-427,190.02","-30,883.35","-458,073.37","-458,073.37",Terceros,2058,"233,680,008.71"
4423,ALLARIA LEDESMA Y CIA S A,401,1401,96367,"-709,550,000.00","-709,550,000.00","-709,550,000.00",0.00,"-709,550,000.00","-709,550,000.00",Terceros,6060,"3,347,914,000.00"
4432,VALCEREAL SA,284,1284,96575,"-449,691.78","-449,691.78","-449,691.78",0.00,"-449,691.78","-449,691.78",Terceros,7257,"134,352,185.84"


### Definir funciones utiles

In [85]:
#def cim_to_id(cim):
#
#    cim = str(cim)
#
#    cimcod = data_cuentas[data_cuentas["CuentaCompensacionCodigo"] == cim]
#    cim = cimcod["CuentaCompensacionID"].to_string(index=False,dtype=False)
#    return(cim)
#
#def mccod_to_id(mc):
#
#    mc = str(mc)
#
#    mccod = data_cuentas[data_cuentas["MiembroCompensadorCodigo"] == mc]
#    mc = mccod["MiembroCompensadorID"].to_string(index=False,dtype=False)
#    mc = mc.split("\n")
#    mc = mc[0]
#    return(mc)
#

### Obtener saldos de las cuentas propias

In [86]:
#dfsaldos = saldo_negativo[["MiembroCompensadorCodigo","CuentaPropiaMc"]]
#dfsaldos = dfsaldos.drop_duplicates()

In [87]:

#listasaldospropios = []
#for i in range(len(dfsaldos)):
#    try:
#        ALYCCOD = (dfsaldos.iloc[i,0])
#        ALYCID = mccod_to_id(ALYCCOD)
#        PROPA = (dfsaldos.iloc[i,1])        
#        FINALIDAD = 2
#        PROPA = cim_to_id(PROPA)
#
#        e = {'SaldoInicial':'sum', 'MargenRequeridoTotal':'sum'}
#
#        tokenobj = {'key': 'value'}
#        URLSALDO = f"https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados?MCId={ALYCID}&CelID={PROPA}"
#        q = requests.get(URLSALDO, json = tokenobj)
#        SALDOALYC  = (q.json())
#        SALDOALYC = pd.DataFrame(SALDOALYC)
#
#        saldoprevia = (SALDOALYC[['MiembroCompensadorID', 'CuentaCompensacionID', "MonedaDescripcion",'FinalidadID',"SaldoInicial","MargenRequeridoTotal"]]) #cambiar Anterior por Total
#        saldoprevia["AP5"] = saldoprevia["SaldoInicial"] + saldoprevia["MargenRequeridoTotal"] #CAMBIAR ANTERIOR POR Total
#        saldoprevia = saldoprevia.groupby("FinalidadID").agg(e)
#        saldoprevia = saldoprevia.reset_index()
#
#        saldoprevia = (saldoprevia[saldoprevia["FinalidadID"] == FINALIDAD])
#        saldoprevia["AP5"] = saldoprevia["SaldoInicial"] + saldoprevia["MargenRequeridoTotal"] #CAMBIAR ANTERIOR POR Total
#
#        calditos = saldoprevia["AP5"].sum()
#        saldodelapropia = calditos
#    except:
#        saldodelapropia = ALYCCOD
#    
#    listasaldospropios.append(saldodelapropia)
    

In [88]:
#dfsaldos["SaldoPropia"] = listasaldospropios



#dfsaldos = dfsaldos[["CuentaPropiaMc","SaldoPropia"]]
#saldo_negativo = saldo_negativo.merge(dfsaldos, on = "CuentaPropiaMc", how = "left")

### ¿Da saldo, recibe saldo? Obtenerlo y mergeralo al df

In [89]:

url = "https://rm4api.testing.acsa:8099/api/cuentas/getcuentacompensacion"
#url = "https://riskzone.anywhereportfolio.com.ar:9099/api/cuentas/getcuentacompensacion"
test = requests.get(url, verify=False)
dasaldo  = (test.json())
dasaldo = pd.DataFrame(dasaldo)
dasaldo = dasaldo[["CuentaCompensacionCodigo","EntidadBursatilDescripcion","DaCreditoIntraCuentaCompensacion","RecibeCreditoIntraCuentaCompensacion","DaCreditoInterCuentaCompensacion","RecibeCreditoInterCuentaCompensacion"]]

saldo_negativo = (saldo_negativo.merge(dasaldo, how='left', on= "CuentaCompensacionCodigo"))


C:\Users\aggonzalez\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rm4api.testing.acsa'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [90]:
saldo_negativo

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,TipoDeCuenta,CuentaPropiaMc,LOP_Propia,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion
0,BARRILLI SOCIEDAD ANONIMA,53,1053,101242,"-22,787,318.95","-2,488,756.95","-2,488,756.95",0.00,"-2,488,756.95","-2,488,756.95",Terceros,101930,0.00,MtR,False,False,False,True
1,AGRINTER CEREALES S R L,526,1526,101274,"-535,275.66","-535,275.66","-535,275.66","2,327.06","-532,948.61","-532,948.61",Terceros,34797,"353,725,230.00",MtR,False,False,False,True
2,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,101276,"-2,312,040.00","-2,312,040.00","-2,312,040.00",0.00,"-2,312,040.00","-2,312,040.00",Terceros,2021,"1,206,339,191.37",MtR,False,False,False,True
3,NASINI SOCIEDAD ANONIMA,218,1218,101567,"1,675,770.84","1,681,804.40","1,681,804.40","-13,444,600.00","-11,762,795.60","-11,762,795.60",Terceros,4729,"365,673.55",MtR,False,False,False,True
4,MAX CAPITAL S.A.,334,1334,101699,"-297,000,000.00","-297,000,000.00","-297,000,000.00",0.00,"-297,000,000.00","-297,000,000.00",Terceros,22558,"182,366,620.00",MtR,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,DIAZ RIGANTI CEREALES SRL,131,1131,96112,"-394,973.50","-394,973.50","-394,973.50",0.00,"-394,973.50","-394,973.50",Terceros,9980,"77,068,000.00",MtR,False,False,False,True
490,ROAGRO SA,42,1042,9633,"-427,190.02","-427,190.02","-427,190.02","-30,883.35","-458,073.37","-458,073.37",Terceros,2058,"233,680,008.71",MtR,False,False,False,True
491,ALLARIA LEDESMA Y CIA S A,401,1401,96367,"-709,550,000.00","-709,550,000.00","-709,550,000.00",0.00,"-709,550,000.00","-709,550,000.00",Terceros,6060,"3,347,914,000.00",MtR,False,False,False,True
492,VALCEREAL SA,284,1284,96575,"-449,691.78","-449,691.78","-449,691.78",0.00,"-449,691.78","-449,691.78",Terceros,7257,"134,352,185.84",MtR,False,False,False,True


### Calcular total CIM y total MC

In [91]:
lista_mc = list(saldo_negativo["MiembroCompensadorCodigo"].drop_duplicates())
lista_cim = list(saldo_negativo["CuentaCompensacionCodigo"].drop_duplicates())

mco=[]
lop_mc=[]
lopno_mc=[]

for mc in lista_mc:
    provisorio_mc = (saldo_negativo[saldo_negativo["MiembroCompensadorCodigo"]==mc])
    lop_verificado_mc = provisorio_mc["LOP_Verificado"].sum()
    lop_noverificado_mc = provisorio_mc["LOP_NoVerificado"].sum()
    lop_mc.append(lop_verificado_mc)
    lopno_mc.append(lop_noverificado_mc)
    mco.append(mc)

listas = [mco,lop_mc,lopno_mc]
dataframe_lops_mc = (pd.DataFrame(listas)).T.rename(columns={0:"MiembroCompensadorCodigo",1:"LOP Verificado MC",2:"LOP No Verificado MC"})
#dataframe_lops_mc = dataframe_lops_mc.rename(columns={"0":"MiembroCompensadorCodigo","1":"LOP Verificado MC","2":"LOP No Verificado MC"},inplace=True)

cim_lop=[]
lop_cim=[]
lopno_cim=[]

for cim in lista_cim:
    provisorio_cim = (saldo_negativo[saldo_negativo["CuentaCompensacionCodigo"]==cim])
    lop_verificado_cim = provisorio_cim["LOP_Verificado"].sum()
    lop_noverificado_cim = provisorio_cim["LOP_NoVerificado"].sum()
    lop_cim.append(lop_verificado_cim)
    lopno_cim.append(lop_noverificado_cim)
    cim_lop.append(cim)
    
listas = [cim_lop,lop_cim,lopno_cim]
dataframe_lops_cim = (pd.DataFrame(listas)).T.rename(columns={0:"CuentaCompensacionCodigo",1:"LOP Verificado CIM",2:"LOP No Verificado CIM"})
#dataframe_lops_mc = dataframe_lops_mc.rename(columns={"0":"MiembroCompensadorCodigo","1":"LOP Verificado MC","2":"LOP No Verificado MC"},inplace=True)

saldo_negativo = saldo_negativo.merge(dataframe_lops_cim, on = "CuentaCompensacionCodigo", how = "left")
saldo_negativo = saldo_negativo.merge(dataframe_lops_mc, on = "MiembroCompensadorCodigo", how = "left")

### CONSEGUIR LOP CIM PROPIA

In [92]:
#lopcimpropia = saldo_negativo[saldo_negativo.TipoDeCuenta == "Propia"]
#lopcimpropia = lopcimpropia.rename(columns = {"CuentaCompensacionCodigo":"CuentaPropiaMc",'LOP_Verificado':'LOP_Verificado_Propia'})
#lop_propia = lopcimpropia[["CuentaPropiaMc","LOP_Verificado_Propia"]]


In [93]:
saldo_negativo

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,...,LOP_Propia,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion,LOP Verificado CIM,LOP No Verificado CIM,LOP Verificado MC,LOP No Verificado MC
0,BARRILLI SOCIEDAD ANONIMA,53,1053,101242,"-22,787,318.95","-2,488,756.95","-2,488,756.95",0.00,"-2,488,756.95","-2,488,756.95",...,0.00,MtR,False,False,False,True,"-2,488,756.95","-2,488,756.95","-2,488,756.95","-2,488,756.95"
1,AGRINTER CEREALES S R L,526,1526,101274,"-535,275.66","-535,275.66","-535,275.66","2,327.06","-532,948.61","-532,948.61",...,"353,725,230.00",MtR,False,False,False,True,"-70,224,792.76","-70,224,792.76","-70,224,792.76","-70,224,792.76"
2,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,101276,"-2,312,040.00","-2,312,040.00","-2,312,040.00",0.00,"-2,312,040.00","-2,312,040.00",...,"1,206,339,191.37",MtR,False,False,False,True,"-642,344,934.47","-642,344,934.47","-642,344,934.47","-642,344,934.47"
3,NASINI SOCIEDAD ANONIMA,218,1218,101567,"1,675,770.84","1,681,804.40","1,681,804.40","-13,444,600.00","-11,762,795.60","-11,762,795.60",...,"365,673.55",MtR,False,False,False,True,"-149,809,275.60","-149,809,275.60","-149,809,275.60","-149,809,275.60"
4,MAX CAPITAL S.A.,334,1334,101699,"-297,000,000.00","-297,000,000.00","-297,000,000.00",0.00,"-297,000,000.00","-297,000,000.00",...,"182,366,620.00",MtR,False,False,False,True,"-379,476,632.62","-379,476,632.62","-412,193,912.62","-412,193,912.62"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,DIAZ RIGANTI CEREALES SRL,131,1131,96112,"-394,973.50","-394,973.50","-394,973.50",0.00,"-394,973.50","-394,973.50",...,"77,068,000.00",MtR,False,False,False,True,"-1,134,719.00","-1,134,719.00","-1,134,719.00","-1,134,719.00"
490,ROAGRO SA,42,1042,9633,"-427,190.02","-427,190.02","-427,190.02","-30,883.35","-458,073.37","-458,073.37",...,"233,680,008.71",MtR,False,False,False,True,"-74,368,970.92","-74,368,970.92","-74,368,970.92","-74,368,970.92"
491,ALLARIA LEDESMA Y CIA S A,401,1401,96367,"-709,550,000.00","-709,550,000.00","-709,550,000.00",0.00,"-709,550,000.00","-709,550,000.00",...,"3,347,914,000.00",MtR,False,False,False,True,"-1,524,796,042.02","-1,524,796,042.02","-1,524,796,042.02","-1,524,796,042.02"
492,VALCEREAL SA,284,1284,96575,"-449,691.78","-449,691.78","-449,691.78",0.00,"-449,691.78","-449,691.78",...,"134,352,185.84",MtR,False,False,False,True,"-20,353,488.57","-20,353,488.57","-2,412,286,436.47","-2,412,286,436.47"


# PROCESO Y CRITERIOS DE ANALISIS

### Primer analisis: encontrar todas las cuentas de neteo pertenencientes a una cim de terceros, comparar LOP CIM con LOP CIM PROPIA y si Propia es mayor, ponerla como cubierta

In [94]:
terceros = pd.DataFrame(saldo_negativo[(saldo_negativo.TipoDeCuenta == "Terceros") & (saldo_negativo.LOP_Verificado < -10)].groupby("CuentaCompensacionCodigo").agg({"LOP_Verificado":"sum","LOP_NoVerificado":"sum","LOP_Propia":"first"}).reset_index())


In [95]:
terceros["ResultadoVerificado"] = terceros.LOP_NoVerificado + terceros.LOP_Propia
terceroscubiertos = terceros[terceros.ResultadoVerificado > 0]

In [96]:
listaterceroscubiertos = list(terceroscubiertos.CuentaCompensacionCodigo)

In [97]:
saldo_negativo

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,...,LOP_Propia,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion,LOP Verificado CIM,LOP No Verificado CIM,LOP Verificado MC,LOP No Verificado MC
0,BARRILLI SOCIEDAD ANONIMA,53,1053,101242,"-22,787,318.95","-2,488,756.95","-2,488,756.95",0.00,"-2,488,756.95","-2,488,756.95",...,0.00,MtR,False,False,False,True,"-2,488,756.95","-2,488,756.95","-2,488,756.95","-2,488,756.95"
1,AGRINTER CEREALES S R L,526,1526,101274,"-535,275.66","-535,275.66","-535,275.66","2,327.06","-532,948.61","-532,948.61",...,"353,725,230.00",MtR,False,False,False,True,"-70,224,792.76","-70,224,792.76","-70,224,792.76","-70,224,792.76"
2,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,101276,"-2,312,040.00","-2,312,040.00","-2,312,040.00",0.00,"-2,312,040.00","-2,312,040.00",...,"1,206,339,191.37",MtR,False,False,False,True,"-642,344,934.47","-642,344,934.47","-642,344,934.47","-642,344,934.47"
3,NASINI SOCIEDAD ANONIMA,218,1218,101567,"1,675,770.84","1,681,804.40","1,681,804.40","-13,444,600.00","-11,762,795.60","-11,762,795.60",...,"365,673.55",MtR,False,False,False,True,"-149,809,275.60","-149,809,275.60","-149,809,275.60","-149,809,275.60"
4,MAX CAPITAL S.A.,334,1334,101699,"-297,000,000.00","-297,000,000.00","-297,000,000.00",0.00,"-297,000,000.00","-297,000,000.00",...,"182,366,620.00",MtR,False,False,False,True,"-379,476,632.62","-379,476,632.62","-412,193,912.62","-412,193,912.62"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,DIAZ RIGANTI CEREALES SRL,131,1131,96112,"-394,973.50","-394,973.50","-394,973.50",0.00,"-394,973.50","-394,973.50",...,"77,068,000.00",MtR,False,False,False,True,"-1,134,719.00","-1,134,719.00","-1,134,719.00","-1,134,719.00"
490,ROAGRO SA,42,1042,9633,"-427,190.02","-427,190.02","-427,190.02","-30,883.35","-458,073.37","-458,073.37",...,"233,680,008.71",MtR,False,False,False,True,"-74,368,970.92","-74,368,970.92","-74,368,970.92","-74,368,970.92"
491,ALLARIA LEDESMA Y CIA S A,401,1401,96367,"-709,550,000.00","-709,550,000.00","-709,550,000.00",0.00,"-709,550,000.00","-709,550,000.00",...,"3,347,914,000.00",MtR,False,False,False,True,"-1,524,796,042.02","-1,524,796,042.02","-1,524,796,042.02","-1,524,796,042.02"
492,VALCEREAL SA,284,1284,96575,"-449,691.78","-449,691.78","-449,691.78",0.00,"-449,691.78","-449,691.78",...,"134,352,185.84",MtR,False,False,False,True,"-20,353,488.57","-20,353,488.57","-2,412,286,436.47","-2,412,286,436.47"


In [98]:


primerfiltrado  = saldo_negativo[~saldo_negativo["CuentaCompensacionCodigo"].isin(listaterceroscubiertos)]

In [99]:
primerfiltrado

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,...,LOP_Propia,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion,LOP Verificado CIM,LOP No Verificado CIM,LOP Verificado MC,LOP No Verificado MC
0,BARRILLI SOCIEDAD ANONIMA,53,1053,101242,"-22,787,318.95","-2,488,756.95","-2,488,756.95",0.00,"-2,488,756.95","-2,488,756.95",...,0.00,MtR,False,False,False,True,"-2,488,756.95","-2,488,756.95","-2,488,756.95","-2,488,756.95"
3,NASINI SOCIEDAD ANONIMA,218,1218,101567,"1,675,770.84","1,681,804.40","1,681,804.40","-13,444,600.00","-11,762,795.60","-11,762,795.60",...,"365,673.55",MtR,False,False,False,True,"-149,809,275.60","-149,809,275.60","-149,809,275.60","-149,809,275.60"
4,MAX CAPITAL S.A.,334,1334,101699,"-297,000,000.00","-297,000,000.00","-297,000,000.00",0.00,"-297,000,000.00","-297,000,000.00",...,"182,366,620.00",MtR,False,False,False,True,"-379,476,632.62","-379,476,632.62","-412,193,912.62","-412,193,912.62"
8,VALCEREAL SA,284,90450,102741,"-1,156,020.00","-1,156,020.00","-1,156,020.00",0.00,"-1,156,020.00","-1,156,020.00",...,"134,352,185.84",MtR,False,False,False,True,"-2,391,932,947.91","-2,391,932,947.91","-2,412,286,436.47","-2,412,286,436.47"
9,BRIO VALORES AGENTE DE LIQUIDACION Y COMPENSAC...,378,1378,102797,"-308,173.35","-308,173.35","-308,173.35",0.00,"-308,173.35","-308,173.35",...,"4,008,000.00",MtR,False,False,False,True,"-6,014,834.74","-6,014,834.74","-6,014,834.74","-6,014,834.74"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,BRIO VALORES AGENTE DE LIQUIDACION Y COMPENSAC...,378,1378,91624,"-394,148.29","-394,148.29","-394,148.29","-3,525,000.00","-3,919,148.29","-3,919,148.29",...,"4,008,000.00",MtR,False,False,False,True,"-6,014,834.74","-6,014,834.74","-6,014,834.74","-6,014,834.74"
470,LEIVA HERMANOS S.A.,352,1352,92106,"-578,787.82","-578,787.82","-578,787.82","-225,000.00","-803,787.82","-803,787.82",...,"40,760.71",MtR,False,False,False,True,"-939,662.61","-939,662.61","-939,662.61","-939,662.61"
476,BRIO VALORES AGENTE DE LIQUIDACION Y COMPENSAC...,378,1378,92871,"-45,000.00","-45,000.00","-45,000.00",0.00,"-45,000.00","-45,000.00",...,"4,008,000.00",MtR,False,False,False,True,"-6,014,834.74","-6,014,834.74","-6,014,834.74","-6,014,834.74"
477,BRIO VALORES AGENTE DE LIQUIDACION Y COMPENSAC...,378,1378,93000,"-45,000.00","-45,000.00","-45,000.00",0.00,"-45,000.00","-45,000.00",...,"4,008,000.00",MtR,False,False,False,True,"-6,014,834.74","-6,014,834.74","-6,014,834.74","-6,014,834.74"


### 2do checkpoint - aca ya sacamos todas las de terceros que estan cubiertas por alyc

In [100]:
primerfiltrado.to_excel("primerfiltrado.xlsx")

PermissionError: [Errno 13] Permission denied: 'primerfiltrado.xlsx'

### seguimos


In [ ]:
df2 = primerfiltrado
df2 = df2[(df2.LOP_Verificado < 0) & (df2.RecibeCreditoIntraCuentaCompensacion == True)].groupby("CuentaCompensacionCodigo").sum()
df2 = df2[["LOP_Verificado"]]
df2.reset_index(inplace=True)

df3 = primerfiltrado[["CuentaCompensacionCodigo", "LOP Verificado CIM"]]
df2 = df2.merge(df3, on = "CuentaCompensacionCodigo", how= "left")
df2["result"] = df2["LOP_Verificado"] + df2["LOP Verificado CIM"]
df4 = df2[(df2.result > 0)]
segundofiltro = list(df4["CuentaCompensacionCodigo"])

segundo_filtro = primerfiltrado[~primerfiltrado["CuentaCompensacionCodigo"].isin(segundofiltro)]

segundofiltro

[]

In [ ]:
df[(df.LOP_Verificado < 0) & (df.MargenRequeridoDelDia < 0)].sort_values("CuentaCompensacionCodigo")

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,...,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion,LOP Verificado CIM,LOP No Verificado CIM,LOP Verificado MC,LOP No Verificado MC,LOP_Verificado_Propia
638,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,67086,"-767,000.00","-767,000.00","-767,000.00",-640.00,"-767,640.00","-767,640.00",...,MtR,False,False,False,True,"-682,257,682.26","-682,257,682.26","512,731,568.11","512,731,568.11","1,194,989,250.37"
393,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,2593,"-1,917,500.00","-1,917,500.00","-1,917,500.00","-1,600.00","-1,919,100.00","-1,919,100.00",...,MtR,False,False,False,True,"-682,257,682.26","-682,257,682.26","512,731,568.11","512,731,568.11","1,194,989,250.37"
334,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,17049,"-6,456,222.50","-6,456,222.50","-6,456,222.50","-3,120.00","-6,459,342.50","-6,459,342.50",...,MtR,False,False,False,True,"-682,257,682.26","-682,257,682.26","512,731,568.11","512,731,568.11","1,194,989,250.37"
320,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,168767,"-2,301,000.00","-2,301,000.00","-2,301,000.00","-1,920.00","-2,302,920.00","-2,302,920.00",...,MtR,False,False,False,True,"-682,257,682.26","-682,257,682.26","512,731,568.11","512,731,568.11","1,194,989,250.37"
838,GRANAR S A COMERCIAL Y FINANCIERA,21,1021,93581,"-43,721,674.95","-43,721,674.95","-43,721,674.95","-98,876.74","-43,820,551.69","-43,820,551.69",...,MtR,False,False,False,True,"-682,257,682.26","-682,257,682.26","512,731,568.11","512,731,568.11","1,194,989,250.37"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812,VALCEREAL SA,284,90450,90723,"-8,628,750.00","-8,628,750.00","-8,628,750.00","-7,200.00","-8,635,950.00","-8,635,950.00",...,MtR,False,False,False,True,"-2,392,950,579.25","-2,392,950,579.25","-2,342,744,658.03","-2,342,744,658.03","65,751,253.73"
813,VALCEREAL SA,284,90450,90725,"-5,752,500.00","-5,752,500.00","-5,752,500.00","-4,800.00","-5,757,300.00","-5,757,300.00",...,MtR,False,False,False,True,"-2,392,950,579.25","-2,392,950,579.25","-2,342,744,658.03","-2,342,744,658.03","65,751,253.73"
815,VALCEREAL SA,284,90450,90730,"-20,157,718.75","-20,157,718.75","-20,157,718.75","-16,820.00","-20,174,538.75","-20,174,538.75",...,MtR,False,False,False,True,"-2,392,950,579.25","-2,392,950,579.25","-2,342,744,658.03","-2,342,744,658.03","65,751,253.73"
771,VALCEREAL SA,284,90450,90662,"-4,880,027.65","-4,880,027.65","-4,880,027.65","-4,754.31","-4,884,781.96","-4,884,781.96",...,MtR,False,False,False,True,"-2,392,950,579.25","-2,392,950,579.25","-2,342,744,658.03","-2,342,744,658.03","65,751,253.73"


In [ ]:
primer_filtro = saldo_negativo[saldo_negativo["TipoDeCuenta"] == "Terceros"]
e = {'LOP_NoVerificado':'sum', 'LOP_Verificado':'sum',"SaldoPropia":"first"}
primerfiltro = primer_filtro.groupby("MiembroCompensadorCodigo").agg(e)
primerfiltro["Resultado"] = primerfiltro["LOP_Verificado"] + primerfiltro["SaldoPropia"].astype(float)
cubiertosporalyc = primerfiltro[primerfiltro["Resultado"] > 0]
listacubiertos = list(cubiertosporalyc.index)
print(listacubiertos)

['125', '131', '133', '136', '146', '155', '162', '163', '168', '21', '218', '22', '26', '269', '284', '313', '325', '327', '334', '335', '339', '345', '350', '352', '360', '366', '378', '401', '405', '409', '411', '413', '42', '46', '466', '475', '526', '527', '530', '536', '62', '92']


In [ ]:
cubiertosporalyc = saldo_negativo[saldo_negativo["MiembroCompensadorCodigo"].isin(listacubiertos) == True]


### AHORA SACAR DEL DATAFRAME ENTERO LOS QUE SI SON CUBIERTOS POR EL ALYC

In [ ]:
NOCUBIERTOS = saldo_negativo[~saldo_negativo.isin(cubiertosporalyc)].dropna()


### test de agrupamiento

In [ ]:
TESTI = NOCUBIERTOS.groupby(["ALyC","CuentaCompensacionCodigo","CuentaNeteoCodigo"]).sum()
#TESTI.to_excel("TestAgrupamiento.xls")
TESTI

,,,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado
ALyC,CuentaCompensacionCodigo,CuentaNeteoCodigo,,,,,,
A G VALORES S.A.,6262,6262,"2,640,768.14","2,639,938.15","2,639,938.15",0.00,"2,639,938.15","2,639,938.15"
ACA VALORES S.A.,146410,146410,0.09,0.09,0.09,0.00,0.09,0.09
AGRICULTORES FEDERADOS ARGENTINOS SOC COOP LTDA,102750,102750,"1,242,516,934.59","1,242,516,934.59","1,242,516,934.59",0.00,"1,242,516,934.59","1,242,516,934.59"
ALCHEMY VALORES S.A.,11205,11205,"364,267,840.00","364,267,840.00","364,267,840.00",0.00,"364,267,840.00","364,267,840.00"
ARCOR S A I C,9605,9605,"155,253,679.05","155,253,679.05","155,253,679.05",0.00,"155,253,679.05","155,253,679.05"
...,...,...,...,...,...,...,...,...
VITERRA ARGENTINA S.A.,142875,142875,"36,116,304.90","36,116,304.90","36,116,304.90",0.00,"36,116,304.90","36,116,304.90"
WIN SECURITIES S.A.,149560,149560,"9,868,600.00","10,047,000.00","10,047,000.00",0.00,"10,047,000.00","10,047,000.00"
YPF SOCIEDAD ANONIMA,13806,13806,"1,038,446,521.54","1,032,546,521.54","1,032,546,521.54","-73,200.00","1,032,473,321.54","1,032,473,321.54"


In [ ]:
NOCUBIERTOS[NOCUBIERTOS["MiembroCompensadorCodigo"] == 274]

,ALyC,MiembroCompensadorCodigo,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado,...,SaldoPropia,EntidadBursatilDescripcion,DaCreditoIntraCuentaCompensacion,RecibeCreditoIntraCuentaCompensacion,DaCreditoInterCuentaCompensacion,RecibeCreditoInterCuentaCompensacion,LOP Verificado CIM,LOP No Verificado CIM,LOP Verificado MC,LOP No Verificado MC


### FILTRAR LAS QUE TENGAN SALDOS NEGATIVOS MAYORES A 100 PESOS

In [ ]:
LOPS = TESTI

In [ ]:
LOPS = LOPS[LOPS["LOP_Verificado"] < -100]
LOPS = LOPS.reset_index()
#LOPS = LOPS.drop("index", axis=1)


In [ ]:
(LOPS)

,ALyC,CuentaCompensacionCodigo,CuentaNeteoCodigo,SaldoInicialAP5,SaldoFinalVerificado,SaldoFinalNoVerificado,MargenRequeridoDelDia,LOP_Verificado,LOP_NoVerificado
0,ASOCIACION DE COOPERATIVAS ARGENTINAS COOP LTDA,1156,142187,"-151,409.50","-151,409.50","-151,409.50",0.00,"-151,409.50","-151,409.50"
1,ASOCIACION DE COOPERATIVAS ARGENTINAS COOP LTDA,1156,142192,"-1,998,642.85","-1,998,642.85","-1,998,642.85",0.00,"-1,998,642.85","-1,998,642.85"
2,ASOCIACION DE COOPERATIVAS ARGENTINAS COOP LTDA,1156,142194,"-146,736.69","-146,736.69","-146,736.69",0.00,"-146,736.69","-146,736.69"
3,ASOCIACION DE COOPERATIVAS ARGENTINAS COOP LTDA,1156,142201,"-560,485.25","-560,485.25","-560,485.25",0.00,"-560,485.25","-560,485.25"
4,ASOCIACION DE COOPERATIVAS ARGENTINAS COOP LTDA,1156,142202,"-455,022.75","-455,022.75","-455,022.75",0.00,"-455,022.75","-455,022.75"
...,...,...,...,...,...,...,...,...,...
117,ROSARIO VALORES SA,1135,5640,"-17,345.46","2,573.54","2,573.54","-1,423,914.12","-1,421,340.58","-1,421,340.58"
118,ROSARIO VALORES SA,1901,155735,0.00,0.00,0.00,-640.00,-640.00,-640.00
119,ROSARIO VALORES SA,1939,169306,"465,952.50","360,490.00","360,490.00","-1,440,925.00","-1,080,435.00","-1,080,435.00"
120,TRANSATLANTICA S.A. BURSATIL,1204,72709,"-34,100.00","-34,100.00","-34,100.00",0.00,"-34,100.00","-34,100.00"


In [ ]:
LOPS_ORDENADOS = LOPS.sort_values(["MiembroCompensadorCodigo","CuentaCompensacionCodigo","CuentaNeteoCodigo"])

KeyError: 'MiembroCompensadorCodigo'

### MERGEAR DFS


In [ ]:
LOPS_ORDENADOS = LOPS_ORDENADOS.merge(dataframe_lops_cim, on = "CuentaCompensacionCodigo", how = "left")
LOPS_ORDENADOS = LOPS_ORDENADOS.merge(dataframe_lops_mc, on = "MiembroCompensadorCodigo", how = "left")
LOPS_ORDENADOS


In [ ]:
cat = pd.read_excel("categorias.xlsx")
#LOPS_ORDENADOS = LOPS_ORDENADOS.merge(cat, on = "MiembroCompensadorCodigo", how = "left")
categorizado = LOPS_ORDENADOS
#categorizado.merge(cat, on = "MiembroCompensadorCodigo", how = "left
categorizado["MiembroCompensadorCodigo"] = categorizado["MiembroCompensadorCodigo"].astype(int)
cat["MiembroCompensadorCodigo"] = cat["MiembroCompensadorCodigo"].astype(int)
LOPS_ORDENADOS = categorizado.merge(cat, on = "MiembroCompensadorCodigo", how = "left")

In [ ]:
def format_float(value):
    return f'{value:,.2f}'

pd.options.display.float_format = format_float
LOPS_ORDENADOS

In [ ]:
ESTADO = []
for i in range(len(LOPS_ORDENADOS)):
    if (LOPS_ORDENADOS.loc[i]["LOP Verificado MC"]) < -1*(LOPS_ORDENADOS.loc[i].tercerlim):
        estado = "LOP C"
    elif (LOPS_ORDENADOS.loc[i]["LOP Verificado MC"]) < -1*(LOPS_ORDENADOS.loc[i].segundolim):
        estado = "LOP B"
    elif (LOPS_ORDENADOS.loc[i]["LOP Verificado MC"]) < -1*(LOPS_ORDENADOS.loc[i].primerlim):
        estado = "LOP A"
    else:
        estado = "NO LOP"
    
    ESTADO.append(estado)


LOPS_ORDENADOS["LOP"] = ESTADO


In [ ]:
LOPS_TEST = LOPS_ORDENADOS.groupby(["ALyC","CuentaCompensacionCodigo","CuentaNeteoCodigo"]).first()
LOPS_TEST
LOPS_TEST.to_excel("LOPS_AGRUPADOS.xlsx")


In [ ]:
LOPS_ORDENADOS.to_excel("LOPS_CASIFINAL.xlsx",index=False)

## Situación LOP
#### Entonces deberiamos

1) Determinar total saldo verificado y no verificado por cuenta de neteo, sólo para
finalidad Márgenes 

2) Determinar margen del día por cuenta de neteo

3) Establecer el LOP verificado y no verificado de cada cuenta de neteo

Cuenta CEL

1) Determina la situación de LOP sumando los LOP verificado (y no verificado por otro lado) de cada cuenta de neteo interna y comparando con los límites de dicha CIM

Cuenta Tercero/CER/AN
1) Determina el LOP verificado y no verificado “total”, sumando los valorescorrespondientes de cada cuenta de neteo interna.
2) Si el total de LOP verificado es menor a 0, y cae dentro de algunos de los límites de LOP, buscamos el remanente de las cuentas propias del ALyC (es decir, los LOP verificados positivos) y en caso de poder, usamos de dicho valor la cantidad necesaria para “salir de LOP”. En este caso, el LOP verificado del AlyC “variaría” (sería un número menor) ya que estaríamos subvencionando a la CIM.
3) Si con el LOP del ALyC no “sale de LOP” no se usa nada y la CIM queda en la situación LOP actual. ¿Cómo usar el remanente de ALyC? Ordena las cuentas CIM por total LOP ascendente (del más negativo al más positivo), y dentro de cada CIM, ordenamos las cuentas de neteo por LOP ascendente. Finalmente, empezamos a utilizar el LOP del ALyC por CIM/Neteo “más negativa”.

Cuenta Propia
1) Determina el LOP verificado y no verificado “total”, sumando los valores correspondientes de cada cuenta de neteo interna.
2) Al LOP verificado y no verificado le “resto” lo que use para compensar otras CIM del ALyC y comparo contra los límites y determino la situación de LOP.
3) El “saldo ALyC” se define como el saldo de la CIM (compenso dentro de la cim propias con los LOP de las cuentas de neteo).



In [110]:
# assign dataframes
data1 = pd.DataFrame([[25, 77.5, 'A'], [30, 60.2, 'B'],
                      [25, 70.7, 'C']],
                     columns=['Students', 'Avg Marks', 'Section'])
  
data2 = pd.DataFrame([[30, 70.2, 'A'], [25, 65.2, 'B'],
                      [35, 77.7, 'C']],
                     columns=['Students', 'Avg Marks', 'Section'])
  
  
# display dataframes
print('Dataframes:')
display(data1)
display(data2)
  
# merge two data frames
print('After merging:')
pd.concat([data1, data2], axis=0)

Dataframes:


,Students,Avg Marks,Section
0,25,77.50,A
1,30,60.20,B
2,25,70.70,C


,Students,Avg Marks,Section
0,30,70.20,A
1,25,65.20,B
2,35,77.70,C


After merging:


,Students,Avg Marks,Section
0,25,77.50,A
1,30,60.20,B
2,25,70.70,C
0,30,70.20,A
1,25,65.20,B
2,35,77.70,C


In [112]:
pd.merge([data1, data2])

TypeError: merge() missing 1 required positional argument: 'right'

: 